In [ ]:
import tensorflow as tf
from diffusers import StableDiffusionPipeline

# Load the pre-trained Stable Diffusion model
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id)

# Define the text prompt
prompt = "A futuristic city with flying cars at sunset"

# Generate the image using PyTorch-based pipeline
generated_image = pipe(prompt).images[0]

# Convert the PIL image to a TensorFlow tensor (optional)
image_tensor = tf.convert_to_tensor(np.array(generated_image), dtype=tf.float32)
image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add batch dimension

# Example: Apply some TensorFlow processing (like resizing the image)
resized_image = tf.image.resize(image_tensor, [512, 512])

# Convert back to image and save
resized_image = tf.cast(resized_image, tf.uint8)
resized_pil_image = tf.keras.preprocessing.image.array_to_img(resized_image[0])
resized_pil_image.save("output_image_tf_resized.png")


2024-09-17 09:11:56.860770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 09:11:56.860802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 09:11:56.861529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 09:11:56.865503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 09:11:57.509655: W tensorflow/compiler/tf2

ImportError: 
StableDiffusionPipeline requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
